In [1]:
from pathlib import Path

from foambryo import dcel_mesh_from_file, infer_forces

fname = "../data/Meshes/Benchmark_original_mesh.rec"


mesh = dcel_mesh_from_file(fname)
dict_tensions, dict_pressure = infer_forces(mesh)

reference_tensions = {}
with Path("reference_tensions.txt").open("r") as f:
    for line in f.readlines():
        l1, l2, tension = line.split(" ")
        reference_tensions[(int(l1), int(l2))] = float(tension)

assert reference_tensions == dict_tensions

reference_pressures = {}
with Path("reference_pressures.txt").open("r") as f:
    for line in f.readlines():
        cell_id, pressure = line.split(" ")
        reference_pressures[int(cell_id)] = float(pressure)

assert reference_pressures.keys() == dict_pressure.keys()
assert reference_pressures == dict_pressure

print("OK !")


OK !
